<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/deepseek_coin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
import time

# 1. Veri Çekme Fonksiyonu
def get_binance_data(symbol, interval='1h', limit=100):
    url = "https://api.binance.com/api/v3/klines"
    params = {'symbol': symbol, 'interval': interval, 'limit': limit}
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

# 2. Strateji Fonksiyonu (DÜZELTİLDİ)
def calculate_devisso_trend(df, zigzag_high_period=10, zigzag_low_period=10, min_movement_percent=0.16,
                            ma_period=20, std_dev_multiplier=2.0, ma_length=10, volume_length=7):
    # ZigZag hesaplamaları (Girinti eklendi)
    df['zigzag_high'] = df['high'].rolling(zigzag_high_period).apply(lambda x: x.idxmax() == 0, raw=False)
    df['zigzag_low'] = df['low'].rolling(zigzag_low_period).apply(lambda x: x.idxmin() == 0, raw=False)

    # Minimum hareket hesaplama
    df['min_movement'] = df['close'] * min_movement_percent / 100

    # Long/Short Mumu sinyalleri
    df['long_mumu'] = (df['zigzag_high'] > df['zigzag_high'].shift()) & (df['close'] - df['close'].shift() >= df['min_movement'])
    df['short_mumu'] = (df['zigzag_low'] < df['zigzag_low'].shift()) & (df['close'].shift() - df['close'] >= df['min_movement'])

    # Bollinger Bantları
    df['ma'] = df['close'].rolling(ma_period).mean()
    df['std_dev'] = df['close'].rolling(ma_period).std()
    df['upper_band'] = df['ma'] + std_dev_multiplier * df['std_dev']
    df['lower_band'] = df['ma'] - std_dev_multiplier * df['std_dev']

    # Sinyal koşulları
    df['long_signal'] = df['long_mumu'] & (df['close'] > df['ma']) & (df['close'] > df['upper_band'])
    df['short_signal'] = df['short_mumu'] & (df['close'] < df['ma']) & (df['close'] < df['lower_band'])

    # Çekme seviyeleri
    df['long_pullback'] = np.where(df['long_mumu'], df['close'], np.nan)
    df['short_pullback'] = np.where(df['short_mumu'], df['close'], np.nan)
    df['long_pullback'].ffill(inplace=True)
    df['short_pullback'].ffill(inplace=True)

    # Ardışık yükseliş/düşüş sayıları
    df['consecutive_up'] = df['long_mumu'].groupby((~df['long_mumu']).cumsum()).cumsum()
    df['consecutive_down'] = df['short_mumu'].groupby((~df['short_mumu']).cumsum()).cumsum()

    # Potansiyel mumlar
    df['short_potential'] = (df['close'] > df['ma']) & (df['close'] < df['long_pullback']) & (df['close'] >= df['upper_band'])
    df['long_potential'] = (df['close'] > df['short_pullback']) & (df['close'] > df['short_pullback'].rolling(2).mean()) & (df['close'] < df['ma'])

    # Hareketli ortalamalar
    df['all_signals'] = np.where(df['long_mumu'], df['low'], np.where(df['short_mumu'], df['high'], np.nan))
    df['ma_all_signals'] = df['all_signals'].rolling(ma_length).mean()
    df['all_signals2'] = np.where(df['short_potential'], df['low'], np.where(df['long_potential'], df['high'], np.nan))
    df['ma_all_signals2'] = df['all_signals2'].rolling(ma_length).mean()
    df['final_ma'] = (df['ma_all_signals'] + df['ma_all_signals2']) / 2

    # Sinyal tespiti
    df['short_crossover'] = df['close'] < df['ma_all_signals2']
    df['long_crossover'] = df['close'] > df['ma_all_signals2']
    df['cross'] = (df['close'] > df['ma_all_signals2']) & (df['close'] > df['ma_all_signals'])

    return df  # Fonksiyonun dönüş değeri

# 3. Coin Tarama Fonksiyonu (Girinti düzeltildi)
def scan_coins(coin_list):
    signals = []
    for coin in coin_list:
        print(f"Processing {coin}...")
        data = get_binance_data(coin)
        result = calculate_devisso_trend(data)

        # Son sinyali kontrol et
        last_row = result.iloc[-1]
        if last_row['long_crossover']:
            signals.append((coin, 'BUY', last_row['close']))
        elif last_row['short_crossover']:
            signals.append((coin, 'SELL', last_row['close']))

    return signals

# 4. Filtreleme Fonksiyonu
def filter_signals(signals, min_volume=100000):
    filtered = []
    for signal in signals:
        coin, action, price = signal
        data = get_binance_data(coin)
        last_volume = data['volume'].iloc[-1]
        if last_volume >= min_volume:
            filtered.append(signal)
    return filtered

# 5. Sürekli Tarama
def continuous_scan(coin_list, interval=3600):
    while True:
        print(f"Scanning at {time.ctime()}...")
        signals = scan_coins(coin_list)
        if signals:
            filtered_signals = filter_signals(signals, min_volume=100000)
            for signal in filtered_signals:
                coin, action, price = signal
                print(f"{coin} - {action} - Fiyat: {price}")
        time.sleep(interval)

# 6. Ana Program
if __name__ == "__main__":
    coin_list = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'ADAUSDT', 'SOLUSDT']
    continuous_scan(coin_list)

Scanning at Mon Feb  3 06:48:09 2025...
Processing BTCUSDT...
Processing ETHUSDT...


<ipython-input-1-94cd3b771027>:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['long_pullback'].ffill(inplace=True)
<ipython-input-1-94cd3b771027>:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.met

Processing BNBUSDT...
Processing ADAUSDT...


<ipython-input-1-94cd3b771027>:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['long_pullback'].ffill(inplace=True)
<ipython-input-1-94cd3b771027>:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.met

Processing SOLUSDT...


<ipython-input-1-94cd3b771027>:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['long_pullback'].ffill(inplace=True)
<ipython-input-1-94cd3b771027>:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.met